In [1]:
#install and download
from IPython.display import display
import pandas as pd
import plotly.express as px
import warnings
from tqdm import notebook
import nltk
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re 
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import numpy as np
import torch
import transformers as ppb
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\laptop\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
warnings.filterwarnings("ignore")

In [3]:
url = 'https://www.kaggle.com/crowdflower/twitter-airline-sentiment/download'

In [4]:
data = pd.read_csv(url)

In [5]:
data['Tone'] = 0

data['Tone'].loc[data['airline_sentiment'] == 'neutral'] = 1
data['Tone'].loc[data['airline_sentiment'] == 'positive'] = 2

In [6]:
data

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,Tone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),1
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),2
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),1
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),0
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,569587686496825344,positive,0.3487,NaN,0.0000,American,NaN,KristenReenders,NaN,0,@AmericanAir thank you we got on a different f...,NaN,2015-02-22 12:01:01 -0800,NaN,NaN,2
14636,569587371693355008,negative,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,@AmericanAir leaving over 20 minutes Late Flig...,NaN,2015-02-22 11:59:46 -0800,Texas,NaN,0
14637,569587242672398336,neutral,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,0,@AmericanAir Please bring American Airlines to...,NaN,2015-02-22 11:59:15 -0800,"Nigeria,lagos",NaN,1
14638,569587188687634433,negative,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,"@AmericanAir you have my money, you change my ...",NaN,2015-02-22 11:59:02 -0800,New Jersey,Eastern Time (US & Canada),0


In [7]:
data['text']=data['text'].apply(str).astype('U')

In [8]:
def lemmatizer(text):
    tokenized = nltk.word_tokenize(text)
    joined = ' '.join(tokenized)
    return joined

In [9]:
notebook.tqdm.pandas() 
data['text'] = data['text'].progress_apply(lemmatizer)

  0%|          | 0/14640 [00:00<?, ?it/s]

In [10]:
def clear_text(row):
    row = re.sub(r"(?:\n|\r)", " ", row)
    row = re.sub(r"[^a-zA-Z ]+", "", row)
    return " ".join(row.split())

data['text'] = data['text'].apply(clear_text)

In [11]:
stopwords = set(stopwords.words('english'))
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\laptop\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
features = data['text']
target = data['Tone']

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.2, random_state=12345,stratify=target)

In [13]:
count_vect = CountVectorizer(stop_words=stopwords)

train_vect = count_vect.fit_transform(features_train)
#valid_vect = count_vect.fit_transform(features_valid)
test_vect = count_vect.transform(features_test)

In [14]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 

train_tf_idf = count_tf_idf.fit_transform(features_train)
#valid_tf_idf = count_vect.fit_transform(features_valid)
test_tf_idf = count_tf_idf.transform(features_test)

In [15]:
def grid_search_LogisticRegression(model, x, y):
        param_grid = [{'C':[1,10],'class_weight':[None]}]
        gs = GridSearchCV(model, param_grid, cv=3,scoring='f1', verbose=1, n_jobs=-1)
        gs.fit(x, y)
        best_estimator = gs.best_estimator_
        return best_estimator

In [16]:
lr = LogisticRegression(solver='liblinear', dual=True, max_iter = 1000)

In [17]:
notebook.tqdm.pandas() 
lr_fit = grid_search_LogisticRegression(lr,train_vect, target_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


In [18]:
predictions_lr_vect = lr_fit.predict(test_vect)
print('Метрика f1 с векторизированными признаками модели Логистическая регрессия {:.10f}'.format(f1_score(target_test, predictions_lr_vect, average='weighted')))

Метрика f1 с векторизированными признаками модели Логистическая регрессия 0.7948162181


In [19]:
notebook.tqdm.pandas() 
lr_tf_idf = grid_search_LogisticRegression(lr,train_tf_idf, target_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


In [20]:
predictions_lr_tf_idf = lr_tf_idf.predict(test_tf_idf)
print('Метрика f1 с TF_IDF-признаками модели Логистическая регрессия {:.10f}'.format(f1_score(target_test, predictions_lr_tf_idf, average='weighted')))

Метрика f1 с TF_IDF-признаками модели Логистическая регрессия 0.7681662237


In [21]:
def grid_search_LGBMClassifier(model, x, y):
        param_grid = [{'max_depth' : [10],
                      'num_leaves' : [10]}]
        gs = GridSearchCV(model, param_grid, cv=3,scoring='f1', verbose=1, n_jobs=-1)
        gs.fit(x, y)
        best_estimator = gs.best_estimator_
        return best_estimator

In [22]:
lgb = LGBMClassifier(boosting_type = 'gbdt')

In [23]:
lgb_tf_idf = grid_search_LGBMClassifier(lgb, train_tf_idf, target_train)
predictions_lgb_tf_idf = lgb_tf_idf.predict(test_tf_idf)
print('Метрика f1 с TF_IDF-признаками {:.10f}'.format(f1_score(target_test, predictions_lgb_tf_idf, average='weighted')))

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Метрика f1 с TF_IDF-признаками 0.7495997787


In [24]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')


tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
features_bert = data['text']
target_bert = data['Tone']

In [26]:
notebook.tqdm.pandas() 
tokenized = features_bert.progress_apply((lambda x: tokenizer.encode(x[:512], add_special_tokens=True)))

  0%|          | 0/14640 [00:00<?, ?it/s]

In [27]:
padded = np.array([i + [0]*(512-len(i)) for i in tokenized.values])

In [28]:
attention_mask = np.where(padded != 0, 1, 0)
padded.shape, attention_mask.shape

((14640, 512), (14640, 512))

In [29]:
batch_size = 10
embeddings = [] # пустой список для хранения эмбеддингов комментариев

for i in notebook.tqdm(range(padded.shape[0] // batch_size)): 
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) # преобразуем данные  в тензоры( многомерные векторы) 
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]) #преобразуем маску 
    #batch_embeddings = model(batch, attention_mask=attention_mask_batch) #передадим модели данные и маску
    
    with torch.no_grad(): #Для ускорения вычисления укажем, что градиенты не нужны: 
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)
    embeddings.append(batch_embeddings[0][:,0,:].numpy()) # преобразуем элементы к типу numpy.array и разрезаем выход для первой позиции во всех последовательностях, и берем все выходы скрытых нейронок (разрезаем этот 3d тензор для того, чтобы получить нужный нам 2d тензор) 


  0%|          | 0/1464 [00:00<?, ?it/s]

In [30]:
features_embeddings = np.concatenate(embeddings)
features_embeddings.shape

(14640, 768)

In [31]:
features_train, features_test, target_train, target_test = train_test_split(
    features_embeddings, target_bert, test_size=0.2, random_state=0,stratify = target_bert)

In [32]:
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(features_train, target_train)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

best parameters:  {'C': 26.315863157894736}
best scrores:  0.7964482024512576


In [33]:
lr_clf = LogisticRegression(C= 5.263252631578947)
lr_clf.fit(features_train, target_train)

LogisticRegression(C=5.263252631578947)

In [34]:
predictions_lr_clf = lr_clf.predict(features_test)
print('Метрика f1 модели DistilBertModel {:.10f}'.format(f1_score(target_test, predictions_lr_clf, average='weighted')))

Метрика f1 модели DistilBertModel 0.7965180857
